In [ ]:
!pip install langchain_core langgraph langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


In [ ]:
def chatbot_node(state: State) -> State:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}


In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()


In [ ]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7, api_key=openai_api_key)

In [ ]:

def test_chatbot(message: str):
    initial_state = {"messages": [HumanMessage(content=message)]}
    result = graph.invoke(initial_state)
    print("🤖 Assistant:", result["messages"][-1].content)

test_chatbot("Hello! My name is Pradip")
test_chatbot("Do you remember my name?")


🤖 Assistant: Hello Pradip! How can I assist you today?
🤖 Assistant: I don't have access to previous interactions, so I don't know your name unless you tell me in this conversation. What would you like me to call you?


In [ ]:
#Adding Memory to the Chatbot

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Compile the graph again with memory enabled
graph_with_memory = graph_builder.compile(checkpointer=memory)


In [ ]:
def chat_with_memory(message: str, thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}
    initial_state = {"messages": [HumanMessage(content=message)]}
    result = graph_with_memory.invoke(initial_state, config)
    print("🤖 Assistant:", result["messages"][-1].content)

# Start a conversation
chat_with_memory("Hi, my name is Pradip", thread_id="thread-1")
chat_with_memory("What's my name?", thread_id="thread-1")


🤖 Assistant: Hello Pradip! How can I assist you today?
🤖 Assistant: Your name is Pradip. How can I help you further?
